In [1]:
RUN_ON_KAGGLE = False
DEBUG = False

In [2]:
%%capture
if RUN_ON_KAGGLE:
    # installing offline dependencies
    !pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
    !cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
    !pip install -U /kaggle/working/sentence-transformers
    !pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

    !pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
    !pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
    !pip install --no-index --no-deps /kaggle/input/llm-whls/datasets-2.14.3-py3-none-any.whl
    !pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl
    
    !pip install "/kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl" --no-index
    !pip install "/kaggle/input/peft-native/peft-0.4.0-py3-none-any.whl" --no-index --find-links="/kaggle/input/peft-native/"
    !pip install "/kaggle/input/bitsandbytes-0410/bitsandbytes-0.41.0-py3-none-any.whl" --no-index
    !pip install "/kaggle/input/accelerate-native/accelerate-0.21.0-py3-none-any.whl" --no-index --find-links="/kaggle/input/accelerate-native/"
    !pip install "/kaggle/input/transformers-native/transformers-4.31.0-py3-none-any.whl" --no-index --find-links="/kaggle/input/transformers-native/"

In [3]:
import os, time
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf
# from __future__ import annotations

from collections.abc import Iterable

import faiss
from faiss import write_index, read_index

from sentence_transformers import SentenceTransformer

import torch
import ctypes
libc = ctypes.CDLL("libc.so.6")

from dataclasses import dataclass
from typing import Optional, Union

import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from torch.utils.data import DataLoader

from scipy.special import softmax


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /home/viktor/miniconda3/envs/torch-env did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


In [4]:
if RUN_ON_KAGGLE:
    SIM_MODEL = '/kaggle/input/sentencetransformers-allminilml6v2/sentence-transformers_all-MiniLM-L6-v2'
else:
    SIM_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 1


FILTER_LEN = 1 if DEBUG else 9
IND_SEARCH = 1 if DEBUG else 7
NUM_SENTENCES_INCLUDE = 1 if DEBUG else 25
CONTEXT_LEN = 1000 if DEBUG else 2305
CONTEXT_LEN_8BIT = 1000 if DEBUG else 4096
VAL_SIZE = 20 if DEBUG else 2000

if RUN_ON_KAGGLE:
    WIKI_PATH = "/kaggle/input/wikipedia-20230701"
else:
    WIKI_PATH = "/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/wikipedia-2023-07-faiss-index"
    
# file_for_local_cv = "/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-llm-science-exam/test.csv"    
file_for_local_cv = "/home/viktor/Documents/kaggle/kaggle_llm/data/data_dumps/more_questions/more_questions_raw_questions_wiki_sci_3_shuffled.csv"    

wiki_files = os.listdir(WIKI_PATH)

In [5]:
def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = FILTER_LEN,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    
    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.values, 
                        df.document_id.values,
                        df.offset.values, 
                        filter_len, 
                        disable_progress_bar)
    return df


def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:

    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df


def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = FILTER_LEN,
               disable_progress_bar: bool = False) -> pd.DataFrame:

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1]-o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0]+offset[0], o[1]+offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)

In [6]:
if RUN_ON_KAGGLE:
    trn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv").drop("id", axis=1)
else:
    trn = pd.read_csv(file_for_local_cv).drop("id", axis=1).sample(n=VAL_SIZE, random_state=42).reset_index(drop=True)
trn.head()

,Unnamed: 0,prompt,A,B,C,D,E,answer,cluster number,round
0,13884,What is the relationship between Adiantum and ...,Adiantum is a variant of AES,Adiantum is a cryptographic message authentica...,Adiantum uses AES for disk encryption,Adiantum is a low-powered mobile device runnin...,Adiantum uses a different construction for the...,C,590,0
1,20379,What is the cause of the painful bump in Osgoo...,Excessive bone growth at the tibial tuberosity,Fluid accumulation in the patellar ligament,Inflammation of the quadriceps muscle,Degeneration of the knee cartilage,Dislocation of the tibiofemoral joint,A,827,1
2,24313,What are the two most common types of tarsal c...,Calcaneo-cuboid and calcaneo-navicular,Talo-cuneiform and talo-tarsal,Navicular-cuboid and calcaneo-tarsal,Talo-calcaneal and talo-navicular,Calcaneo-navicular and talo-calcaneal,E,988,0
3,22258,What is the significance of dietary restrictio...,Dietary restrictions can worsen the symptoms o...,Dietary restrictions can completely cure ocula...,Dietary restrictions have no impact on the sym...,Dietary restrictions may reduce or eliminate t...,Dietary restrictions only impact the symptoms ...,D,902,0
4,16046,What is the purpose of isolating Parvibaculum ...,To remove it from wastewater treatment plants ...,To improve the aesthetics of activated sludge,To study its metabolic abilities and potential...,To increase the production of linear alkylbenz...,To enhance the growth of other bacteria in the...,C,668,0


# Inference

In [7]:
test_df = pd.read_csv("test_context.csv")
test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
# test_df["prompt"] = test_df["context"].apply(lambda x: x[:CONTEXT_LEN]) + " #### " +  test_df["prompt"]

In [8]:

options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
  
    
    first_sentence = [ "[CLS] " + example['context'] ] * 5
    second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']
    tokenized_example = tokenizer(first_sentence, 
                                    second_sentences, 
                                    truncation='only_first', 
                                    max_length=CONTEXT_LEN, 
                                    add_special_tokens=False)        
    if "answer" in example:
        tokenized_example["label"] = option_to_index[example["answer"]]
    
    return tokenized_example
            
    # first_sentence = [example['prompt']] * 5
    # second_sentence = []
    # for option in options:
    #     second_sentence.append(example[option])
    
    # tokenized_example = tokenizer(first_sentence, second_sentence, truncation='only_first')
    # tokenized_example['label'] = option_to_index[example['answer']]
    # return tokenized_example
    
def preprocess_for_8bit(example):
  
    
    first_sentence = [ "[CLS] " + example['context'] ] * 5
    second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']
    tokenized_example = tokenizer(first_sentence, 
                                    second_sentences, 
                                    truncation='only_first', 
                                    max_length=CONTEXT_LEN_8BIT, 
                                    add_special_tokens=False)        
    if "answer" in example:
        tokenized_example["label"] = option_to_index[example["answer"]]
    
    return tokenized_example
            
    # first_sentence = [example['prompt']] * 5
    # second_sentence = []
    # for option in options:
    #     second_sentence.append(example[option])
    
    # tokenized_example = tokenizer(first_sentence, second_sentence, truncation='only_first')
    # tokenized_example['label'] = option_to_index[example['answer']]
    # return tokenized_example

In [9]:
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        # label_name = "label" if 'label' in features[0].keys() else 'labels'
        # labels = [feature.pop(label_name) for feature in features]
        # batch_size = len(features)
        # num_choices = len(features[0]['input_ids'])
        # flattened_features = [
        #     [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        # ]
        # flattened_features = sum(flattened_features, [])
        
        # batch = self.tokenizer.pad(
        #     flattened_features,
        #     padding=self.padding,
        #     max_length=self.max_length,
        #     pad_to_multiple_of=self.pad_to_multiple_of,
        #     return_tensors='pt',
        # )
        # batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        # return batch
        
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [
                {k: v[i] for k, v in feature.items() if k not in ("context", "__index_level_0__", "label", "labels")}
                for i in range(num_choices)
            ] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        # _max_length = max([len(x["input_ids"]) for x in flattened_features])
        # print(f"{_max_length = }")

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        if "label" in features[0].keys() or "labels" in features[0].keys():
            label_name = "label" if "label" in features[0].keys() else "labels"
            labels = [feature.pop(label_name) for feature in features]
            batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch
        
        

# longformer (kaggle)

In [10]:
from transformers import LongformerTokenizer, LongformerForMultipleChoice

if RUN_ON_KAGGLE:
    tokenizer = LongformerTokenizer.from_pretrained("/kaggle/input/longformer-race-model/longformer_qa_model")
    model = LongformerForMultipleChoice.from_pretrained("/kaggle/input/longformer-race-model/longformer_qa_model").cuda()
else:
    tokenizer = LongformerTokenizer.from_pretrained("/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/longformer-race-model/longformer_qa_model")
    model = LongformerForMultipleChoice.from_pretrained("/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/longformer-race-model/longformer_qa_model").cuda()
    

In [11]:
if RUN_ON_KAGGLE:
    !cp /kaggle/input/datasets-wheel/datasets-2.14.4-py3-none-any.whl /kaggle/working
    !pip install  /kaggle/working/datasets-2.14.4-py3-none-any.whl
    !cp /kaggle/input/backup-806-viktor/util_openbook.py .

In [12]:
import os
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf

from collections.abc import Iterable

import faiss
from faiss import write_index, read_index

from sentence_transformers import SentenceTransformer

import torch
import ctypes
libc = ctypes.CDLL("libc.so.6")

from dataclasses import dataclass
from typing import Optional, Union

import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from torch.utils.data import DataLoader


def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 3,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Main helper function to process documents from the EMR.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param document_type: String denoting the document type to be processed
    :param document_sections: List of sections for a given document type to process
    :param split_sentences: Flag to determine whether to further split sections into sentences
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """

    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.values,
                        df.document_id.values,
                        df.offset.values,
                        filter_len,
                        disable_progress_bar)
    return df


def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Obtains the sections of the imaging reports and returns only the
    selected sections (defaults to FINDINGS, IMPRESSION, and ADDENDUM).

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `offset`
    """
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df


def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 3,
               disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Split a document into sentences. Can be used with `sectionize_documents`
    to further split documents into more manageable pieces. Takes in offsets
    to ensure that after splitting, the sentences can be matched to the
    location in the original documents.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param offsets: Iterable tuple of the start and end indices
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents),
                                              disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1] - o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0] + offset[0], o[1] + offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)


def get_contexts(RUN_ON_KAGGLE, VAL_SIZE, file_for_local_cv):
    if RUN_ON_KAGGLE:
        SIM_MODEL = '/kaggle/input/sentencetransformers-allminilml6v2/sentence-transformers_all-MiniLM-L6-v2'
    else:
        SIM_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'
        
    DEVICE = 0
    MAX_LENGTH = 384
    BATCH_SIZE = 16

    if RUN_ON_KAGGLE:
        WIKI_PATH = "/kaggle/input/wikipedia-20230701"
    else:
        WIKI_PATH = "/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/wikipedia-2023-07-faiss-index"
    wiki_files = os.listdir(WIKI_PATH)


    if RUN_ON_KAGGLE:
        trn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv").drop("id", axis=1)
    else:
        trn = pd.read_csv(file_for_local_cv,index_col=0).sample(n=VAL_SIZE, random_state=42).reset_index(drop=True)

    model = SentenceTransformer(SIM_MODEL, device='cuda')
    model.max_seq_length = MAX_LENGTH
    model = model.half()

    if RUN_ON_KAGGLE:
        sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")
    else:
        sentence_index = read_index("/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/wikipedia-2023-07-faiss-index/wikipedia_202307.index")

    # prompt_embeddings = model.encode(trn.prompt.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
    prompt_embeddings = model.encode(
        trn.apply(lambda row: f"{row['prompt']}\n{row['A']}\n{row['B']}\n{row['C']}\n{row['D']}\n{row['E']}",
                  axis=1).values,
        batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)

    prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
    _ = gc.collect()

    # Get the top 20 pages that are likely to contain the topic of interest
    search_score, search_index = sentence_index.search(prompt_embeddings, 20)

    # Save memory - delete sentence_index since it is no longer necessary
    del sentence_index
    del prompt_embeddings
    _ = gc.collect()
    libc.malloc_trim(0)

    if RUN_ON_KAGGLE:
        df = pd.read_parquet("/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet", columns=['id', 'file'])
    else:
        df = pd.read_parquet("/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/wikipedia-2023-07-faiss-index/wiki_2023_index.parquet", columns=['id', 'file'])

    # Get the article and associated file location using the index
    wikipedia_file_data = []

    for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
        scr_idx = idx
        _df = df.loc[scr_idx].copy()
        _df['prompt_id'] = i
        wikipedia_file_data.append(_df)
    wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
    wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(
        ['file', 'id']).reset_index(drop=True)

    # Save memory - delete df since it is no longer necessary
    del df
    _ = gc.collect()
    libc.malloc_trim(0)

    # Get the full text data
    wiki_text_data = []

    for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
        _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file'] == file]['id'].tolist()]
        _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text', 'title'])

        _df_temp = _df[_df['id'].isin(_id)].copy()
        del _df
        _ = gc.collect()
        libc.malloc_trim(0)
        wiki_text_data.append(_df_temp)
    wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
    _ = gc.collect()

    # Parse documents into sentences
    processed_wiki_text_data = process_documents(wiki_text_data.text.values, wiki_text_data.id.values)

    # Get embeddings of the wiki text data
    wiki_data_embeddings = model.encode(processed_wiki_text_data.text,
                                        batch_size=BATCH_SIZE,
                                        device=DEVICE,
                                        show_progress_bar=True,
                                        convert_to_tensor=True,
                                        normalize_embeddings=True)  # .half()
    wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()

    _ = gc.collect()

    # Combine all answers
    trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)

    # Search using the prompt and answers to guide the search
    trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']

    question_embeddings = model.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE,
                                       show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
    question_embeddings = question_embeddings.detach().cpu().numpy()

    # Parameter to determine how many relevant sentences to include
    NUM_SENTENCES_INCLUDE = 6

    # List containing just Context
    contexts = []

    for r in tqdm(trn.itertuples(), total=len(trn)):

        prompt_id = r.Index

        prompt_indices = processed_wiki_text_data[processed_wiki_text_data['document_id'].isin(
            wikipedia_file_data[wikipedia_file_data['prompt_id'] == prompt_id]['id'].values)].index.values

        if prompt_indices.shape[0] > 0:
            prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
            prompt_index.add(wiki_data_embeddings[prompt_indices])

            context = ""

            # Get the top matches
            ss, ii = prompt_index.search(question_embeddings, NUM_SENTENCES_INCLUDE)
            for _s, _i in zip(ss[prompt_id], ii[prompt_id]):
                context += processed_wiki_text_data.loc[prompt_indices]['text'].iloc[_i] + " "
        contexts.append(context)

    trn['context'] = contexts

    trn[["prompt", "context", "A", "B", "C", "D", "E"]].to_csv("./test_context.csv", index=False)


# def generate_openbook_output(RUN_ON_KAGGLE):
#     test_df = pd.read_csv("test_context.csv")
#     test_df.index = list(range(len(test_df)))
#     test_df['id'] = list(range(len(test_df)))
#     test_df["prompt"] = test_df["context"].apply(lambda x: x[:1750]) + " #### " + test_df["prompt"]
#     test_df['answer'] = 'A'
#     if RUN_ON_KAGGLE:
#         model_dir = "/kaggle/input/llm-science-run-context-2"
#     else:
#         model_dir = "/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/llm-science-run-context-2"
#     tokenizer = AutoTokenizer.from_pretrained(model_dir)
#     model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
#     model.eval()

#     # We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
#     options = 'ABCDE'
#     indices = list(range(5))

#     option_to_index = {option: index for option, index in zip(options, indices)}
#     index_to_option = {index: option for option, index in zip(options, indices)}

#     def preprocess(example):
#         # The AutoModelForMultipleChoice class expects a set of question/answer pairs
#         # so we'll copy our question 5 times before tokenizing
#         first_sentence = [example['prompt']] * 5
#         second_sentence = []
#         for option in options:
#             second_sentence.append(example[option])
#         # Our tokenizer will turn our text into token IDs BERT can understand
#         tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
#         tokenized_example['label'] = option_to_index[example['answer']]
#         return tokenized_example

#     tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
#     tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
#     data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
#     test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)

#     test_predictions = []
#     for batch in test_dataloader:
#         for k in batch.keys():
#             batch[k] = batch[k].cuda()
#         with torch.no_grad():
#             outputs = model(**batch)
#         test_predictions.append(outputs.logits.cpu().detach())

#     test_predictions = torch.cat(test_predictions)

#     predictions_as_ids = np.argsort(-test_predictions, 1)

#     predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
#     # predictions_as_answer_letters[:3]

#     predictions_as_string = test_df['prediction'] = [
#         ' '.join(row) for row in predictions_as_answer_letters[:, :3]
#     ]

#     submission = test_df[['id', 'prediction']]
#     submission.to_csv('submission_backup.csv', index=False)


In [13]:
import pickle

get_contexts(RUN_ON_KAGGLE, VAL_SIZE, file_for_local_cv)
# generate_openbook_output(RUN_ON_KAGGLE)

import gc
gc.collect()

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

Batches:   0%|          | 0/726 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

95

In [14]:
import pandas as pd

In [15]:
if RUN_ON_KAGGLE:
    !cp -r /kaggle/input/stem-wiki-cohere-no-emb /kaggle/working
    !cp -r /kaggle/input/all-paraphs-parsed-expanded /kaggle/working/

In [16]:
from datasets import load_dataset, load_from_disk

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
import unicodedata


def SplitList(mylist, chunk_size):
    return [mylist[offs:offs+chunk_size] for offs in range(0, len(mylist), chunk_size)]

def get_relevant_documents_parsed(df_test):
    df_chunk_size=600
    if RUN_ON_KAGGLE:
        paraphs_parsed_dataset = load_from_disk("/kaggle/working/all-paraphs-parsed-expanded")
    else:
        paraphs_parsed_dataset = load_from_disk("/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/all-paraphs-parsed-expanded")
    
    modified_texts = paraphs_parsed_dataset.map(lambda example:
                                             {'temp_text':
                                              f"{example['title']} {example['section']} {example['text']}".replace('\n'," ").replace("'","")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_test.shape[0], df_chunk_size)):
        df_test_ = df_test.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_test_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         paraphs_parsed_dataset[idx.item()]["title"],
                         paraphs_parsed_dataset[idx.item()]["text"],
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles



def get_relevant_documents(df_test):
    df_chunk_size=800
    if RUN_ON_KAGGLE:
        cohere_dataset_filtered = load_from_disk("/kaggle/working/stem-wiki-cohere-no-emb")
    else:
        cohere_dataset_filtered = load_from_disk("/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/wiki-stem-cohere")
    modified_texts = cohere_dataset_filtered.map(lambda example:
                                             {'temp_text':
                                              unicodedata.normalize("NFKD", f"{example['title']} {example['text']}").replace('"',"")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_test.shape[0], df_chunk_size)):
        df_test_ = df_test.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_test_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         cohere_dataset_filtered[idx.item()]["title"],
                         unicodedata.normalize("NFKD", cohere_dataset_filtered[idx.item()]["text"]),
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles



def retrieval(df_test, modified_texts):
    
    corpus_df_test = df_test.apply(lambda row:
                                     f'{row["prompt"]}\n{row["prompt"]}\n{row["prompt"]}\n{row["A"]}\n{row["B"]}\n{row["C"]}\n{row["D"]}\n{row["E"]}',
                                     axis=1).values
    vectorizer1 = TfidfVectorizer(ngram_range=(1,2),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words)
    vectorizer1.fit(corpus_df_test)
    vocab_df_test = vectorizer1.get_feature_names_out()
    vectorizer = TfidfVectorizer(ngram_range=(1,2),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words,
                                 vocabulary=vocab_df_test)
    vectorizer.fit(modified_texts[:500000])
    corpus_tf_idf = vectorizer.transform(corpus_df_test)
    
    print(f"length of vectorizer vocab is {len(vectorizer.get_feature_names_out())}")

    chunk_size = 100000
    top_per_chunk = 10
    top_per_query = 10

    all_chunk_top_indices = []
    all_chunk_top_values = []

    for idx in tqdm(range(0, len(modified_texts), chunk_size)):
        wiki_vectors = vectorizer.transform(modified_texts[idx: idx+chunk_size])
        temp_scores = (corpus_tf_idf * wiki_vectors.T).toarray()
        chunk_top_indices = temp_scores.argpartition(-top_per_chunk, axis=1)[:, -top_per_chunk:]
        chunk_top_values = temp_scores[np.arange(temp_scores.shape[0])[:, np.newaxis], chunk_top_indices]

        all_chunk_top_indices.append(chunk_top_indices + idx)
        all_chunk_top_values.append(chunk_top_values)

    top_indices_array = np.concatenate(all_chunk_top_indices, axis=1)
    top_values_array = np.concatenate(all_chunk_top_values, axis=1)
    
    merged_top_scores = np.sort(top_values_array, axis=1)[:,-top_per_query:]
    merged_top_indices = top_values_array.argsort(axis=1)[:,-top_per_query:]
    articles_indices = top_indices_array[np.arange(top_indices_array.shape[0])[:, np.newaxis], merged_top_indices]
    
    return articles_indices, merged_top_scores


def prepare_answering_input(
        tokenizer, 
        question,  
        options,   
        context,   
        max_seq_length=4096,
    ):
    c_plus_q   = context + ' ' + tokenizer.bos_token + ' ' + question
    c_plus_q_4 = [c_plus_q] * len(options)
    tokenized_examples = tokenizer(
        c_plus_q_4, options,
        max_length=max_seq_length,
        padding="longest",
        truncation=False,
        return_tensors="pt",
    )
    input_ids = tokenized_examples['input_ids'].unsqueeze(0)
    attention_mask = tokenized_examples['attention_mask'].unsqueeze(0)
    example_encoded = {
        "input_ids": input_ids.to(model.device.index),
        "attention_mask": attention_mask.to(model.device.index),
    }
    return example_encoded

In [18]:
stop_words = ['each', 'you', 'the', 'use', 'used',
                  'where', 'themselves', 'nor', "it's", 'how', "don't", 'just', 'your',
                  'about', 'himself', 'with', "weren't", 'hers', "wouldn't", 'more', 'its', 'were',
                  'his', 'their', 'then', 'been', 'myself', 're', 'not',
                  'ours', 'will', 'needn', 'which', 'here', 'hadn', 'it', 'our', 'there', 'than',
                  'most', "couldn't", 'both', 'some', 'for', 'up', 'couldn', "that'll",
                  "she's", 'over', 'this', 'now', 'until', 'these', 'few', 'haven',
                  'of', 'wouldn', 'into', 'too', 'to', 'very', 'shan', 'before', 'the', 'they',
                  'between', "doesn't", 'are', 'was', 'out', 'we', 'me',
                  'after', 'has', "isn't", 'have', 'such', 'should', 'yourselves', 'or', 'during', 'herself',
                  'doing', 'in', "shouldn't", "won't", 'when', 'do', 'through', 'she',
                  'having', 'him', "haven't", 'against', 'itself', 'that',
                  'did', 'theirs', 'can', 'those',
                  'own', 'so', 'and', 'who', "you've", 'yourself', 'her', 'he', 'only',
                  'what', 'ourselves', 'again', 'had', "you'd", 'is', 'other',
                  'why', 'while', 'from', 'them', 'if', 'above', 'does', 'whom',
                  'yours', 'but', 'being', "wasn't", 'be']

In [19]:
if RUN_ON_KAGGLE:
    df_valid = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
else:
    df_valid = pd.read_csv(file_for_local_cv).sample(n=VAL_SIZE, random_state=42).reset_index(drop=True)


In [20]:
retrieved_articles_parsed = get_relevant_documents_parsed(df_valid)
gc.collect()

Loading cached processed dataset at /home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/all-paraphs-parsed-expanded/cache-d6a95ad6d2289009_*_of_00002.arrow


  0%|          | 0/1 [00:00<?, ?it/s]

/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'd', 'doesn', 'don', 'isn', 'll', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won'] not in stop_words.
  warnings.warn(


length of vectorizer vocab is 911


  0%|          | 0/22 [00:00<?, ?it/s]

38

In [21]:
df_valid

,Unnamed: 0,id,prompt,A,B,C,D,E,answer,cluster number,round
0,13884,13988,What is the relationship between Adiantum and ...,Adiantum is a variant of AES,Adiantum is a cryptographic message authentica...,Adiantum uses AES for disk encryption,Adiantum is a low-powered mobile device runnin...,Adiantum uses a different construction for the...,C,590,0
1,20379,20507,What is the cause of the painful bump in Osgoo...,Excessive bone growth at the tibial tuberosity,Fluid accumulation in the patellar ligament,Inflammation of the quadriceps muscle,Degeneration of the knee cartilage,Dislocation of the tibiofemoral joint,A,827,1
2,24313,24459,What are the two most common types of tarsal c...,Calcaneo-cuboid and calcaneo-navicular,Talo-cuneiform and talo-tarsal,Navicular-cuboid and calcaneo-tarsal,Talo-calcaneal and talo-navicular,Calcaneo-navicular and talo-calcaneal,E,988,0
3,22258,22394,What is the significance of dietary restrictio...,Dietary restrictions can worsen the symptoms o...,Dietary restrictions can completely cure ocula...,Dietary restrictions have no impact on the sym...,Dietary restrictions may reduce or eliminate t...,Dietary restrictions only impact the symptoms ...,D,902,0
4,16046,16155,What is the purpose of isolating Parvibaculum ...,To remove it from wastewater treatment plants ...,To improve the aesthetics of activated sludge,To study its metabolic abilities and potential...,To increase the production of linear alkylbenz...,To enhance the growth of other bacteria in the...,C,668,0
5,17785,17902,In what field do Latin rectangles have applica...,Mathematics and number theory,Computer science and data visualization,Statistics and the design of experiments,Chemistry and molecular biology,Physics and quantum mechanics,C,730,1
6,5117,5151,What is the role of acetolactate decarboxylase?,To cleave carbon-carbon bonds in the acetoacet...,To convert (R)-2-acetoin to (S)-2-hydroxy-2-me...,To catalyze the synthesis of (S)-2-hydroxy-2-m...,To participate in butanoate metabolism and c5-...,To catalyze the conversion of (S)-2-hydroxy-2-...,E,279,0
7,11895,11975,What are the main fields of computational engi...,Numerical analysis and scientific simulation,Structural mechanics and acoustics,Computational biology and chemistry,Electromagnetics and thermodynamics,Elasticity and fluid mechanics,E,524,1
8,11941,12021,What is the significance of the name 'mycofact...,It is closely related to the biosynthesis of s...,"It is derived from 'Mycobacterium', predicting...",It predicts the co-occurrence of enzymes as a ...,It is derived from the peptide sequence of myc...,It indicates its involvement in redox pathways,C,525,1
9,10163,10243,What is the term used to describe AIPL1 mutati...,Impaired vision or blindness,The earlier onset of LCA,Increased severity of LCA,Recessive LCA,Impaired electroretinogram,C,459,1


In [22]:
retrieved_articles = get_relevant_documents(df_valid)
gc.collect()

Loading cached processed dataset at /home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/wiki-stem-cohere/cache-e94d488c6798573e_*_of_00002.arrow


  0%|          | 0/1 [00:00<?, ?it/s]

/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'd', 'doesn', 'don', 'isn', 'll', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won'] not in stop_words.
  warnings.warn(


length of vectorizer vocab is 911


  0%|          | 0/28 [00:00<?, ?it/s]

38

In [23]:
from transformers import LongformerTokenizer, LongformerForMultipleChoice

if RUN_ON_KAGGLE:
    tokenizer = LongformerTokenizer.from_pretrained("/kaggle/input/longformer-race-model/longformer_qa_model")
    model = LongformerForMultipleChoice.from_pretrained("/kaggle/input/longformer-race-model/longformer_qa_model").cuda()
else:
    tokenizer = LongformerTokenizer.from_pretrained("/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/longformer-race-model/longformer_qa_model")
    model = LongformerForMultipleChoice.from_pretrained("/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-datasets/longformer-race-model/longformer_qa_model").cuda()
    

In [24]:
predictions = []

test_predictions_longformer_1 = []
test_predictions_longformer_2 = []
longformer_contexts = []


for index in tqdm(range(df_valid.shape[0])):
    row = df_valid.iloc[index]
    # question is 'prompt'
    question = row['prompt']
    options = [row['A'], row['B'], row['C'], row['D'], row['E']]
    context1 = f"{retrieved_articles[index][-4][2]}\n{retrieved_articles[index][-3][2]}\n{retrieved_articles[index][-2][2]}\n{retrieved_articles[index][-1][2]}"
    context2 = f"{retrieved_articles_parsed[index][-3][2]}\n{retrieved_articles_parsed[index][-2][2]}\n{retrieved_articles_parsed[index][-1][2]}"
    
    longformer_contexts.append(context1)
    
    inputs1 = prepare_answering_input(
        tokenizer=tokenizer, question=question,
        options=options, context=context1,
        )
    inputs2 = prepare_answering_input(
        tokenizer=tokenizer, question=question,
        options=options, context=context2,
        )
    
    with torch.no_grad():
        outputs1 = model(**inputs1)    
        losses1 = -outputs1.logits[0].detach().cpu().numpy()
        probability1 = torch.softmax(torch.tensor(-losses1), dim=-1)
        
    with torch.no_grad():
        outputs2 = model(**inputs2)
        losses2 = -outputs2.logits[0].detach().cpu().numpy()
        probability2 = torch.softmax(torch.tensor(-losses2), dim=-1)
        
    
    test_predictions_longformer_1.append(probability1.numpy().astype(np.float16))
    test_predictions_longformer_2.append(probability2.numpy().astype(np.float16))
    


  0%|          | 0/20 [00:00<?, ?it/s]

# combine predictions (and find best combination on local CV )

In [32]:
preds_dict = {
    'test_predictions_longformer_1':np.stack(test_predictions_longformer_1),
}
# save preds_dict as pickle
with open('preds_dict_longformer.pickle', 'wb') as handle:
    pickle.dump(preds_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [33]:

import pickle

# load preds_dict_longformer.pickle as preds_dict
with open('preds_dict_longformer.pickle', 'rb') as handle:
    preds_dict = pickle.load(handle)
    
preds_dict

weights = np.ones(len(preds_dict))

predictions_overall = np.sum([preds_val*weight for preds_val, weight in zip(preds_dict.values(), weights)], axis=0)
predictions_overall = np.argsort(-predictions_overall)[:,:3]
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_overall]
df_valid['prediction'] = [' '.join(pred) for pred in predictions_as_answer_letters]

def compute_map3(preds, actuals):
    correct_answers = [0, 0, 0]
    for pred, actual in zip(preds, actuals):
        if pred[0] == actual:
            correct_answers[0] += 1
        if pred[1] == actual:
            correct_answers[1] += 1./2
        if pred[2] == actual:
            correct_answers[2] += 1./3
    
    n_total = len(actuals)
    map3 = np.sum(correct_answers) / n_total
    
    return map3

if not RUN_ON_KAGGLE and 'answer' in df_valid.columns:
    val_df = pd.read_csv(file_for_local_cv,index_col=0).sample(n=VAL_SIZE, random_state=42).reset_index(drop=True)
    
    val_df['A'] = val_df['A'].map(str)
    val_df['B'] = val_df['B'].map(str)
    val_df['C'] = val_df['C'].map(str)
    val_df['D'] = val_df['D'].map(str)
    val_df['E'] = val_df['E'].map(str)
    val_df['answer'] = val_df['answer'].map(str)

    val_df = val_df.reset_index(drop=True)
    
    
    weights = np.ones(len(preds_dict))
    predictions_overall = np.sum([preds_val*weight for preds_val, weight in zip(preds_dict.values(), weights)], axis=0)
    predictions_overall = np.argsort(-predictions_overall)[:,:3]
    predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_overall]



    map3_score = compute_map3(predictions_as_answer_letters, val_df['answer'].values)


    best_map3_score = map3_score
    print(weights)
    print(map3_score)

    
    # best_map3_score = 0
    # for i in tqdm(range(10000)):
    #     weights = np.random.uniform(0, 1, len(preds_dict))
    #     predictions_overall = np.sum([preds_val*weight for preds_val in zip(preds_dict.values(), weights)], axis=0)
    #     predictions_overall = np.argsort(-predictions_overall)[:,:3]
    #     predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_overall]


    
    #     map3_score = compute_map3(predictions_as_answer_letters, val_df['answer'].values)
    
    #     if map3_score > best_map3_score:
    #         best_map3_score = map3_score
    #         print(weights)
    #         print(map3_score)
    
    # print(map3_score)
    
    

[1.]
0.675


Experiments:

- all models: 0.8

In [27]:
submission = df_valid[['id', 'prediction']]
submission.to_csv('submission.csv', index=False)

pd.read_csv('submission.csv').head(10)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 submission = df_valid[['id', 'prediction']]                                                  │
│   2 submission.to_csv('submission.csv', index=False)                                             │
│   3                                                                                              │
│   4 pd.read_csv('submission.csv').head(10)                                                       │
│                                                                                                  │
│ /home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/pandas/core/frame.py:3767 in  │
│ __getitem__                                                                                      │
│                                                                                                  │
│    3764 │   │   else:                                                                            │
│    3765 │   │   │   if is_iterator(key):                                                         │
│    3766 │   │   │   │   key = list(key)                                                          │
│ ❱  3767 │   │   │   indexer = self.columns._get_indexer_strict(key, "columns")[1]                │
│    3768 │   │                                                                                    │
│    3769 │   │   # take() does not accept boolean indexers                                        │
│    3770 │   │   if getattr(indexer, "dtype", None) == bool:                                      │
│                                                                                                  │
│ /home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/pandas/core/indexes/base.py:5 │
│ 876 in _get_indexer_strict                                                                       │
│                                                                                                  │
│   5873 │   │   else:                                                                             │
│   5874 │   │   │   keyarr, indexer, new_indexer = self._reindex_non_unique(keyarr)               │
│   5875 │   │                                                                                     │
│ ❱ 5876 │   │   self._raise_if_missing(keyarr, indexer, axis_name)                                │
│   5877 │   │                                                                                     │
│   5878 │   │   keyarr = self.take(indexer)                                                       │
│   5879 │   │   if isinstance(key, Index):                                                        │
│                                                                                                  │
│ /home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/pandas/core/indexes/base.py:5 │
│ 938 in _raise_if_missing                                                                         │
│                                                                                                  │
│   5935 │   │   │   │   raise KeyError(f"None of [{key}] are in the [{axis_name}]")               │
│   5936 │   │   │                                                                                 │
│   5937 │   │   │   not_found = list(ensure_index(key)[missing_mask.nonzero()[0]].unique())       │
│ ❱ 5938 │   │   │   raise KeyError(f"{not_found} not in index")                                   │
│   5939 │                                                                                         │
│   5940 │   @overload                                                                             │
│   5941 │   def _get_indexer_non_comparable(                